## Working with LLM Programatically: LangChain with Simple Chain Pattern
This example demonstrates how to configure a LangChain setup that connects to a OpenAI compatible LLM private endpoint using environment based credentials, and then prompts the model to generate a short overview of “VMware Tanzu Hub.”

### Step 0: Set the working directory for the project

In [10]:
# set the working directory for the project
%cd /home/vcap/app/cf-jupyterlab-workshop

/home/vcap/app/cf-jupyterlab-workshop


### Step 1: Import the dependencies

In [11]:
import sys, os
import requests
import json
import httpx
import warnings
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain
from langchain_openai import ChatOpenAI
from tanzu_utils import CFGenAIService
warnings.filterwarnings('ignore')

### Step 2: Set up the OpenAI API credentials

In [12]:
# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
chat_service = CFGenAIService("tanzu-gpt-oss-120b")

# List available models
models = chat_service.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": chat_service.api_base + "/openai/v1",
    "api_key": chat_service.api_key,
    "model_name": models[0]["name"]
}

- openai/gpt-oss-120b (capabilities: CHAT, TOOLS)


### Step 3: Initialize the LLM

In [13]:
# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

### Step 4: Create a prompt template, create a chain, and run the chain

In [14]:
# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    """Provide a concise 5-line overview of {topic}.
    Focus on its purpose, key capabilities, core components,
    and how it integrates within the VMware Tanzu ecosystem.
    Write in a clear, professional tone suitable for a technical summary."""
)
# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

# Define the topic
topic = "VMware Tanzu Hub"

# Run the chain
response = chain.run(topic)
print(response)

- **Purpose:** VMware Tanzu Hub serves as a unified control plane that streamlines the provisioning, governance, and lifecycle management of modern applications across multiple Kubernetes clusters.  
- **Key Capabilities:** Centralized cluster registration, automated policy enforcement, multi‑cluster observability, and secure, role‑based access to workloads and infrastructure.  
- **Core Components:** Tanzu Hub API server, the Tanzu Hub UI dashboard, a policy engine (OPA‑based), and an integration layer for VMware Tanzu Mission Control, Tanzu Kubernetes Grid, and Tanzu Observability.  
- **Ecosystem Integration:** It connects seamlessly with Tanzu Application Platform, Tanzu Build Service, and Tanzu Service Mesh, providing a single source of truth for cataloged services and shared configuration.  
- **Benefit:** By consolidating management of disparate Kubernetes environments, Tanzu Hub accelerates developer velocity while maintaining enterprise‑grade security and compliance across the